# Analytic optimization example

In this example, we consider a simple optimization problem to illustrate GEMSEO interface and MMA optimization library integration.


## Imports

In [ ]:
from gemseo.algos.design_space import DesignSpace
from gemseo.api import configure_logger
from gemseo.api import create_discipline
from gemseo.api import create_scenario
from gemseo.api import execute_post
from numpy import array
from numpy import ones

configure_logger()

## Define objective function, constraints and its gradients.

The objective function:

In [ ]:
def obj_func(x=0.0, y=0.0):
    """The objective function."""
    f = (x - 1.0) ** 2 + (y - 1.0) ** 2
    return f


def d_obj_func(x=0.0, y=0.0):
    """The objective function jacobian."""
    jac = array([2.0 * (x[0] - 1.0), 2 * (y[0] - 1.0)])
    return jac

The constraint function:

In [ ]:
def cstr_func(x=0.0, y=0.0):
    """The inequality constraint function."""
    g = x + y - 1.0
    return g


def d_cstr_func(x=0.0, y=0.0):
    """The inequality constraint function jacobian."""
    jac = ones((1, 2))
    return jac

The GEMSEO autopy disciplines:

In [ ]:
disc1 = create_discipline("AutoPyDiscipline", py_func=obj_func, py_jac=d_obj_func)
disc2 = create_discipline("AutoPyDiscipline", py_func=cstr_func, py_jac=d_cstr_func)

## Define the design space

In [ ]:
ds = DesignSpace()
ds.add_variable("x", l_b=0.0, u_b=1.0, value=0.1)
ds.add_variable("y", l_b=0.0, u_b=1.0, value=0.1)

## Define the scenario

In [ ]:
scenario = create_scenario(
    disciplines=[disc1, disc2],
    formulation="DisciplinaryOpt",
    objective_name="f",
    design_space=ds,
)
scenario.add_constraint("g", "ineq")

## Define the scenario execution options.

In [ ]:
ineq_tol = 1e-5
convergence_tol = 1e-8
options = {
    "algo": "MMA",
    "max_iter": 50,
    "algo_options": {
        "tol": 1e-5,
        "normalize_design_space": True,
        "ineq_tolerance": ineq_tol,
        "xtol_rel": convergence_tol,
        "xtol_abs": convergence_tol,
        "ftol_rel": convergence_tol,
        "ftol_abs": convergence_tol,
    },
}

## Execute the optimization algorithm.

In [ ]:
scenario.execute(options)
opt = scenario.formulation.opt_problem.solution.x_opt

## Post-process the results

In [ ]:
execute_post(scenario, "OptHistoryView", show=True, save=False)